In [2]:
import time
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
# download and load all models
preload_models()

/opt/conda/envs/bark/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/bark/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [28]:
a = time.time()
text_prompt = "[upbeat music] ♪ Got an idea To do what [upbeat music] I feel Taking a trip Just for a trip [upbeat music] ♪"
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("bark_generation3.wav", SAMPLE_RATE, audio_array)
  
# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)
print(time.time() - a , "seconds")

100%|██████████| 35/35 [00:21<00:00,  1.60it/s]


30.15314030647278 seconds


In [4]:
help(generate_audio)

Help on function generate_audio in module bark.api:

generate_audio(text: str, history_prompt: Union[Dict, str, NoneType] = None, text_temp: float = 0.7, waveform_temp: float = 0.7, silent: bool = False, output_full: bool = False)
    Generate audio array from input text.
    
    Args:
        text: text to be turned into audio
        history_prompt: history choice for audio cloning
        text_temp: generation temperature (1.0 more diverse, 0.0 more conservative)
        waveform_temp: generation temperature (1.0 more diverse, 0.0 more conservative)
        silent: disable progress bar
        output_full: return full generation to be used as a history prompt
    
    Returns:
        numpy audio array at sample frequency 24khz



## long form generation

In [14]:
script = """
text_prompt = "♪ [Thumping bass ESG style] Got an idea To do what I feel [Graham Coxon Skronk] Taking a trip Just for a trip [Wonky Synth] ♪"

""".replace("\n", " ").strip()

In [15]:
script

'text_prompt = "♪ [Thumping bass ESG style] Got an idea To do what I feel [Graham Coxon Skronk] Taking a trip Just for a trip [Wonky Synth] ♪"'

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
sentences = nltk.sent_tokenize(script)


In [18]:
sentences

['text_prompt = "♪ [Thumping bass ESG style] Got an idea To do what I feel [Graham Coxon Skronk] Taking a trip Just for a trip [Wonky Synth] ♪"']

In [20]:
from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

In [21]:
GEN_TEMP = 0.6
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

100%|██████████| 34/34 [00:21<00:00,  1.61it/s]


In [13]:
write_wav("bark_generation-long.wav", SAMPLE_RATE, audio_array)

In [22]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)